<div id="container" style="position:relative;">
<div style="float:left"><h1>  Capstone Projects - William Nankap </h1></div>
<div style="position:relative; float:right"><img style="height:65px" src ="https://drive.google.com/uc?export=view&id=1EnB0x-fdqMp6I5iMoEBBEuxB_s7AmE2k" />
</div>
</div>

Bienvenue sur notre notebook. Nous allons travailler sur un ensemble de donnees contenant des messages echanges par un groupe de personne (etudiant, contributeur) sur la platforme Gitter.

Le dataset a ete importe via ce lien https://www.kaggle.com/datasets/freecodecamp/all-posts-public-main-chatroom?select=freecodecamp_casual_chatroom_anon.csv.

Plan de travail: https://docs.google.com/presentation/d/10xtITocBaQgZYtDw1yWe_FzQPgR7Je6EkNRv8IJVCPM/edit?usp=sharing



# Summary

   #### Introduction
   #### 1. Data Loading
   #### 2. Data cleaning
   #### 3. Data exploring
   #### 4. Model preparation
   #### 5. Regression model
   #### 6. NLP

# Introduction

Nous allons nous fixer 2 objectifs business:
   - Categoriser les differents mots presents dans notre dataset
   - Creer un modele de prediction du prochain mot

C'est sur ces objectifs que nous baserons notre recherche et notre exploration du dataset.

In [1]:
# We import all the librairies need for exploration, visualization and modeling

import pandas as pd
import numpy as np

#
# plotting
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns


from scipy import stats
from statsmodels.stats import multicomp
from scipy.stats import pearsonr
import statsmodels.api as sm


# from sklearn.linear_model import LinearRegression


# 1. Data Loading

After downloanding the .zip file from the link share above, we get 2 .csv files:
 - freecodecamp_casual_chatroom.csv
 - freecodecamp_casual_chatroom_anon.csv

We will load both to see what it's inside, and try to understand and plan what to do with each file.

In [2]:
# Loading the file freecodecamp_casual_chatroom.csv into a dataframe
freecodecamp_casual = pd.read_csv('archive/freecodecamp_casual_chatroom.csv', low_memory=False)

#What's the shape of freecodecamp_casual
freecodecamp_casual.shape

(5057400, 26)

This file have `5057400 rows & 26 columns`.

In [3]:
# #What are the dtype for each columns
# freecodecamp_casual.info()

In [4]:
#We have 25columns and we want to see all of them clearly, so we set an option to pandas to display all columns
pd.set_option('display.max_columns', None)

#We take a look on the top 5 rows
# freecodecamp_casual.head()

**Important:** 
  - The column `sent` look like containing date inside but it's define as an object, we need to change that into **datetime.**
  - The column `Unnamed: 0` look like an index, we will explore it to see if we can remove and use the initial index or not.
  - We see some column containing `null values`, we will explore them more to decide what can be do for this null values.
  - We see some column containing `[]`, we will explore them to decide how to deal with that. The mentions column contain array, we will investigate to see how to deal with that, maybe create dummy variable?
  - The column `v` is not clearly define how to work with that?
  - Some columns look like containing duplicated information:
      - `html` vs `sent`
      - `fromUser.url` vs `fromUser.username`
      - `fromUser.v` vs `v`
      - `fromUser.avatarUrlMedium` vs `fromUser.avatarUrlSmall`

We will deal with all these questions/notes in part 2 of the work. But for a first observation, this dataset look like containing a lot of message coming from discussion between group of individu. The information which may be important for us could be the `text`, the `userid`, the `date`, the `status`, the `unread/read`, the `readers`.

Before to go deeper into these dataset, we need to check to 2nd dataset.

In [5]:
# # Loading the file freecodecamp_casual_chatroom.csv into a dataframe
# freecodecamp_anon = pd.read_csv('archive/freecodecamp_casual_chatroom_anon.csv', low_memory=False)

# #What's the shape of freecodecamp_casual
# freecodecamp_anon.shape

This file have `5057400 rows` & `19 columns.`
It's the same number of rows as the first file but they differ only on column number.

In [6]:
# #What are the dtype for each columns
# freecodecamp_anon.info()

It's almost the same columns, we just notice 8 columns missing and 1 new column present.
Missing columns:
  - fromUser: float64
  - fromUser.avatarUrl: object
  - fromUser.avatarUrlMedium: object 
  - fromUser.avatarUrlSmall: object
  - fromUser.displayName: object
  - fromUser.url: object
  - fromUser.username: object
  - mentions: object 

New column:
  - Index: int64
  
We will continue the exploration and see how to deal with that.

In [7]:
# #We have 19columns and we want to see all of them clearly, so we set an option to pandas to display all columns
# pd.set_option('display.max_columns', None)

# #We take a look on the top 5 rows
# freecodecamp_anon.head()

In [8]:
#We take a look on the last 5 rows from freecodecamp_casual
# freecodecamp_casual.tail()

In [9]:
#We take a look on the last 5 rows from freecodecamp_casual_anon
# freecodecamp_anon.tail()

Look like the 5 first rows on `freecodecamp_casual_anon` file are really similary to the 5 first rows of the `freecodecamp_casual` file. Same thing with the last 5 rows for each. 

We will go to the assumptions that the second file is a copy of the firts but with less column.
We will work on the first file `freecodecamp_casual.csv`.

# 2. Data Cleaning 

We are working on the file: **`freecodecamp_casual.csv`.** In the first overview of the file we saw:
   - Null value inside
   - Duplicates columns
   - Error dtype

Our `cleaning` step will consist to deal with that and everything else to have a proper dataset reading for work.

#### Loading the file freecodecamp_casual_chatroom.csv into a dataframe
freecodecamp_casual = pd.read_csv('archive/freecodecamp_casual_chatroom.csv', low_memory=False)

In [10]:
#An overview on our dataset
freecodecamp_casual.head()

,Unnamed: 0,editedAt,fromUser,fromUser.avatarUrl,fromUser.avatarUrlMedium,fromUser.avatarUrlSmall,fromUser.displayName,fromUser.gv,fromUser.id,fromUser.removed,fromUser.staff,fromUser.url,fromUser.username,fromUser.v,html,id,issues,mentions,meta,readBy,sent,status,text,unread,urls,v
0,0,NaN,NaN,NaN,https://avatars2.githubusercontent.com/u/21996...,https://avatars2.githubusercontent.com/u/21996...,Charles Watson,3.0,546fd99cdb8155e6700d6ec8,NaN,NaN,/sircharleswatson,sircharleswatson,1.0,no legumes either,54a4804fb2b2de4b786502bf,[],[],[],22,2014-12-31T23:01:35.647Z,NaN,no legumes either,False,[],1
1,1,NaN,NaN,NaN,https://avatars0.githubusercontent.com/u/10361...,https://avatars0.githubusercontent.com/u/10361...,janetwalters008,NaN,54a47e0cdb8155e6700e486e,NaN,NaN,/janetwalters008,janetwalters008,NaN,That bullet proof coffee sounds insane.,54a4809b9597baaa7dceceb0,[],[],[],21,2014-12-31T23:02:51.600Z,NaN,That bullet proof coffee sounds insane.,False,[],1
2,2,NaN,NaN,NaN,https://avatars0.githubusercontent.com/u/10361...,https://avatars0.githubusercontent.com/u/10361...,janetwalters008,NaN,54a47e0cdb8155e6700e486e,NaN,NaN,/janetwalters008,janetwalters008,NaN,That guy has huge eyes.,54a480b29597baaa7dceceb1,[],[],[],21,2014-12-31T23:03:14.221Z,NaN,That guy has huge eyes.,False,[],1
3,3,NaN,NaN,NaN,https://avatars2.githubusercontent.com/u/21996...,https://avatars2.githubusercontent.com/u/21996...,Charles Watson,3.0,546fd99cdb8155e6700d6ec8,NaN,NaN,/sircharleswatson,sircharleswatson,1.0,"<span data-link-type=""mention"" data-screen-nam...",54a480b89597baaa7dceceb2,[],"[{'userIds': [], 'userId': '54a47e0cdb8155e670...",[],22,2014-12-31T23:03:20.182Z,NaN,@janetwalters008 It is. but it works. some peo...,False,[],1
4,4,NaN,NaN,NaN,https://avatars0.githubusercontent.com/u/42194...,https://avatars0.githubusercontent.com/u/42194...,Paul Gilliam,3.0,54a44bbbdb8155e6700e47de,NaN,NaN,/phgilliam,phgilliam,1.0,They guy that came up with the idea is kind of...,54a480ca9597baaa7dceceb5,[],[],[],22,2014-12-31T23:03:38.388Z,NaN,They guy that came up with the idea is kind of...,False,[],1


### Change dtype

We will change the **dtype** of the `sent` column from `object` to `datetime`.

In [11]:
#Transform the dtype to datetime
freecodecamp_casual['sent'] = pd.to_datetime(freecodecamp_casual['sent'])
freecodecamp_casual['sent']

0         2014-12-31 23:01:35.647000+00:00
1         2014-12-31 23:02:51.600000+00:00
2         2014-12-31 23:03:14.221000+00:00
3         2014-12-31 23:03:20.182000+00:00
4         2014-12-31 23:03:38.388000+00:00
                        ...               
5057395   2017-12-09 11:25:08.516000+00:00
5057396   2017-12-09 11:25:09.738000+00:00
5057397   2017-12-09 11:25:20.024000+00:00
5057398   2017-12-09 11:25:23.849000+00:00
5057399   2017-12-09 11:25:51.849000+00:00
Name: sent, Length: 5057400, dtype: datetime64[ns, UTC]

In [12]:
#sanity check
freecodecamp_casual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5057400 entries, 0 to 5057399
Data columns (total 26 columns):
 #   Column                    Dtype              
---  ------                    -----              
 0   Unnamed: 0                int64              
 1   editedAt                  object             
 2   fromUser                  float64            
 3   fromUser.avatarUrl        object             
 4   fromUser.avatarUrlMedium  object             
 5   fromUser.avatarUrlSmall   object             
 6   fromUser.displayName      object             
 7   fromUser.gv               float64            
 8   fromUser.id               object             
 9   fromUser.removed          object             
 10  fromUser.staff            object             
 11  fromUser.url              object             
 12  fromUser.username         object             
 13  fromUser.v                float64            
 14  html                      object             
 15  id             

We confirm that the `sent` column has the correct **dtype.**

It's look like the column `editedAt` refer to a date, so we'll look inside and deal with that.

In [13]:
#An overview on editedAt column
freecodecamp_casual['editedAt'].describe()

count                       126394
unique                      125175
top       2017-03-03T00:58:58.714Z
freq                             2
Name: editedAt, dtype: object

In [14]:
#How many null value inside editedAt column
freecodecamp_casual['editedAt'].isna().sum()

4931006

Look like `editedAt` contain date but contain `4931006 null value ~ 97.5%` too, we will change the dtype from `object` to `datetime` and manage the null value later.

In [15]:
#Transform the dtype to datetime
freecodecamp_casual['editedAt'] = pd.to_datetime(freecodecamp_casual['editedAt'])
freecodecamp_casual['editedAt']

0         NaT
1         NaT
2         NaT
3         NaT
4         NaT
           ..
5057395   NaT
5057396   NaT
5057397   NaT
5057398   NaT
5057399   NaT
Name: editedAt, Length: 5057400, dtype: datetime64[ns, UTC]

In [16]:
#sanity check
freecodecamp_casual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5057400 entries, 0 to 5057399
Data columns (total 26 columns):
 #   Column                    Dtype              
---  ------                    -----              
 0   Unnamed: 0                int64              
 1   editedAt                  datetime64[ns, UTC]
 2   fromUser                  float64            
 3   fromUser.avatarUrl        object             
 4   fromUser.avatarUrlMedium  object             
 5   fromUser.avatarUrlSmall   object             
 6   fromUser.displayName      object             
 7   fromUser.gv               float64            
 8   fromUser.id               object             
 9   fromUser.removed          object             
 10  fromUser.staff            object             
 11  fromUser.url              object             
 12  fromUser.username         object             
 13  fromUser.v                float64            
 14  html                      object             
 15  id             

We confirm the change from `object` to `datetime` for the column `editedAt`.

We have finish to deal with **dtype** to change, then we go to the next step by treating `null value` now.

### Deal with Null Value

We will treat `null value` first by each column.

In [17]:
#An overview on each column, which column contain null value?
i = 0
for col in freecodecamp_casual.columns:
    sum_isna = freecodecamp_casual[col].isna().sum()
    if sum_isna > 0:
        i += 1
        print(f'{col} have {sum_isna} null value')
print(f'\n We have {i} columns which contains null value')

editedAt have 4931006 null value
fromUser have 5057400 null value
fromUser.avatarUrl have 1828000 null value
fromUser.avatarUrlMedium have 7 null value
fromUser.avatarUrlSmall have 7 null value
fromUser.displayName have 11 null value
fromUser.gv have 31837 null value
fromUser.id have 7 null value
fromUser.removed have 5053990 null value
fromUser.staff have 5057373 null value
fromUser.url have 7 null value
fromUser.username have 7 null value
fromUser.v have 796555 null value
html have 19845 null value
status have 5041092 null value
text have 19559 null value

 We have 16 columns which contains null value


We will clean null value from these column:
   - editedAt -  4931006 null value 
   - fromUser -  5057400 null value
   - fromUser.avatarUrl  -  1828000 null value
   - fromUser.avatarUrlMedium  -  7 null value
   - fromUser.avatarUrlSmall  -  7 null value
   - fromUser.displayName - 11 null value
   - fromUser.gv - 31837 null value
   - fromUser.id - 7 null value
   - fromUser.removed - 5053990 null value
   - fromUser.staff - 5057373 null value
   - fromUser.url - 7 null value
   - fromUser.username - 7 null value
   - fromUser.v - 796555 null value
   - html - 19845 null value
   - status - 5041092 null value
   - text - 19559 null value

In [18]:
#First we make a copy of our dataset, and we will work on it.
clean_freecodecamp_casual = freecodecamp_casual.copy()

#sanity check
clean_freecodecamp_casual.head()

,Unnamed: 0,editedAt,fromUser,fromUser.avatarUrl,fromUser.avatarUrlMedium,fromUser.avatarUrlSmall,fromUser.displayName,fromUser.gv,fromUser.id,fromUser.removed,fromUser.staff,fromUser.url,fromUser.username,fromUser.v,html,id,issues,mentions,meta,readBy,sent,status,text,unread,urls,v
0,0,NaT,NaN,NaN,https://avatars2.githubusercontent.com/u/21996...,https://avatars2.githubusercontent.com/u/21996...,Charles Watson,3.0,546fd99cdb8155e6700d6ec8,NaN,NaN,/sircharleswatson,sircharleswatson,1.0,no legumes either,54a4804fb2b2de4b786502bf,[],[],[],22,2014-12-31 23:01:35.647000+00:00,NaN,no legumes either,False,[],1
1,1,NaT,NaN,NaN,https://avatars0.githubusercontent.com/u/10361...,https://avatars0.githubusercontent.com/u/10361...,janetwalters008,NaN,54a47e0cdb8155e6700e486e,NaN,NaN,/janetwalters008,janetwalters008,NaN,That bullet proof coffee sounds insane.,54a4809b9597baaa7dceceb0,[],[],[],21,2014-12-31 23:02:51.600000+00:00,NaN,That bullet proof coffee sounds insane.,False,[],1
2,2,NaT,NaN,NaN,https://avatars0.githubusercontent.com/u/10361...,https://avatars0.githubusercontent.com/u/10361...,janetwalters008,NaN,54a47e0cdb8155e6700e486e,NaN,NaN,/janetwalters008,janetwalters008,NaN,That guy has huge eyes.,54a480b29597baaa7dceceb1,[],[],[],21,2014-12-31 23:03:14.221000+00:00,NaN,That guy has huge eyes.,False,[],1
3,3,NaT,NaN,NaN,https://avatars2.githubusercontent.com/u/21996...,https://avatars2.githubusercontent.com/u/21996...,Charles Watson,3.0,546fd99cdb8155e6700d6ec8,NaN,NaN,/sircharleswatson,sircharleswatson,1.0,"<span data-link-type=""mention"" data-screen-nam...",54a480b89597baaa7dceceb2,[],"[{'userIds': [], 'userId': '54a47e0cdb8155e670...",[],22,2014-12-31 23:03:20.182000+00:00,NaN,@janetwalters008 It is. but it works. some peo...,False,[],1
4,4,NaT,NaN,NaN,https://avatars0.githubusercontent.com/u/42194...,https://avatars0.githubusercontent.com/u/42194...,Paul Gilliam,3.0,54a44bbbdb8155e6700e47de,NaN,NaN,/phgilliam,phgilliam,1.0,They guy that came up with the idea is kind of...,54a480ca9597baaa7dceceb5,[],[],[],22,2014-12-31 23:03:38.388000+00:00,NaN,They guy that came up with the idea is kind of...,False,[],1


#### Column: fromUser - 5057400 null value

It's look like there is nothing inside the `fromUser` column, because we have as much `null value` as `number of rows`.

In [19]:
#Verification of assumptions
clean_freecodecamp_casual['fromUser'].value_counts()

Series([], Name: fromUser, dtype: int64)

There is nothing inside this column so we choose to delete it.

In [20]:
#Delete fromUser column
clean_freecodecamp_casual = clean_freecodecamp_casual.drop(columns=['fromUser'])

#sanity check
clean_freecodecamp_casual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5057400 entries, 0 to 5057399
Data columns (total 25 columns):
 #   Column                    Dtype              
---  ------                    -----              
 0   Unnamed: 0                int64              
 1   editedAt                  datetime64[ns, UTC]
 2   fromUser.avatarUrl        object             
 3   fromUser.avatarUrlMedium  object             
 4   fromUser.avatarUrlSmall   object             
 5   fromUser.displayName      object             
 6   fromUser.gv               float64            
 7   fromUser.id               object             
 8   fromUser.removed          object             
 9   fromUser.staff            object             
 10  fromUser.url              object             
 11  fromUser.username         object             
 12  fromUser.v                float64            
 13  html                      object             
 14  id                        object             
 15  issues         

#### Column where number of null value < 100
Those columns are:
- fromUser.avatarUrlMedium - 7 null value
- fromUser.avatarUrlSmall - 7 null value
- fromUser.displayName - 11 null value
- fromUser.id - 7 null value
- fromUser.url - 7 null value
- fromUser.username - 7 null value

We will see what is inside, check if it's possible to put a random value oderwise delete them.

In [21]:
#fromUser.avatarUrlMedium 
clean_freecodecamp_casual[clean_freecodecamp_casual['fromUser.avatarUrlMedium'].isna()]

,Unnamed: 0,editedAt,fromUser.avatarUrl,fromUser.avatarUrlMedium,fromUser.avatarUrlSmall,fromUser.displayName,fromUser.gv,fromUser.id,fromUser.removed,fromUser.staff,fromUser.url,fromUser.username,fromUser.v,html,id,issues,mentions,meta,readBy,sent,status,text,unread,urls,v
101,101,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hello world!,54a4a0de29aec36b6af1c0ef,[],[],[],16,2015-01-01 01:20:30.403000+00:00,NaN,hello world!,False,[],1
106,106,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Thanks <span data-link-type=""mention"" data-scr...",54a4a1499597baaa7dced08d,[],"[{'userIds': [], 'userId': '546fc9f1db8155e670...",[],17,2015-01-01 01:22:17.160000+00:00,NaN,Thanks @QuincyLarson :),False,[],1
110,110,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I&#39;ve found Free Code Camp in reddit and I ...,54a4a22b9597baaa7dced099,[],[],[],17,2015-01-01 01:26:03.133000+00:00,NaN,I've found Free Code Camp in reddit and I alre...,False,[],1
126,126,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<span data-link-type=""mention"" data-screen-nam...",54a4a39229aec36b6af1c110,[],"[{'userIds': [], 'userId': '546fc9f1db8155e670...",[],20,2015-01-01 01:32:02.517000+00:00,NaN,@QuincyLarson No problem. I work as a native m...,False,[],1
22983,22983,2015-01-15 21:45:35.640000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54b834fa723506dd0c0edf6b,[],[],[],21,2015-01-15 21:45:30.474000+00:00,NaN,NaN,False,[],2
22988,22988,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<span data-link-type=""mention"" data-screen-nam...",54b83523573df0f923453022,[],"[{'userIds': [], 'userId': '54aaf442db8155e670...",[],19,2015-01-15 21:46:11.499000+00:00,NaN,@ESOComputing just testing how Gitter looks in...,False,[],1
83004,83004,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quincy&#39;s streaming on Twitch right now if ...,54e57fe0c8e106d730ca19d8,[],[],[],17,2015-02-19 06:17:04.992000+00:00,NaN,Quincy's streaming on Twitch right now if you ...,False,[{'url': 'http://www.twitch.tv/freecodecamp'}],1


Look like the rows where `fromUser.avatarUrlMedium` are `null` doesn't have any interesting data inside, we can delete them to clean that.

In [22]:
#Get all rows index
ind = clean_freecodecamp_casual[clean_freecodecamp_casual['fromUser.avatarUrlMedium'].isna()].index.values.tolist()
print(ind)

#deleting all these index in my datasaet
# for i in ind:
clean_freecodecamp_casual = clean_freecodecamp_casual.drop(ind)

#sanity check
clean_freecodecamp_casual[clean_freecodecamp_casual['fromUser.avatarUrlMedium'].isna()]

[101, 106, 110, 126, 22983, 22988, 83004]


,Unnamed: 0,editedAt,fromUser.avatarUrl,fromUser.avatarUrlMedium,fromUser.avatarUrlSmall,fromUser.displayName,fromUser.gv,fromUser.id,fromUser.removed,fromUser.staff,fromUser.url,fromUser.username,fromUser.v,html,id,issues,mentions,meta,readBy,sent,status,text,unread,urls,v


In [23]:
#sanity check
clean_freecodecamp_casual['fromUser.avatarUrlMedium'].isna().sum()

0

We confirm that there no more `null value` inside the `fromUser.avatarUrlMedium` column.

In [24]:
#fromUser.avatarUrlSmall 
clean_freecodecamp_casual['fromUser.avatarUrlSmall'].isna().sum()

0

We confirm that there no more `null value` inside the `fromUser.avatarUrlSmall` column.

In [25]:
#fromUser.id 
clean_freecodecamp_casual['fromUser.id'].isna().sum()

0

We confirm that there no more `null value` inside the `fromUser.id` column.

In [26]:
#fromUser.url 
clean_freecodecamp_casual['fromUser.id'].isna().sum()

0

We confirm that there no more `null value` inside the `fromUser.url` column.

In [27]:
#fromUser.username 
clean_freecodecamp_casual['fromUser.username'].isna().sum()

0

We confirm that there no more `null value` inside the `fromUser.username` column.

In [28]:
#fromUser.displayName 
clean_freecodecamp_casual['fromUser.displayName'].isna().sum()

4

In [29]:
#Which rows containing this null value
clean_freecodecamp_casual[clean_freecodecamp_casual['fromUser.displayName'].isna()]

,Unnamed: 0,editedAt,fromUser.avatarUrl,fromUser.avatarUrlMedium,fromUser.avatarUrlSmall,fromUser.displayName,fromUser.gv,fromUser.id,fromUser.removed,fromUser.staff,fromUser.url,fromUser.username,fromUser.v,html,id,issues,mentions,meta,readBy,sent,status,text,unread,urls,v
149462,149462,NaT,NaN,https://avatars1.githubusercontent.com/u/14760...,https://avatars1.githubusercontent.com/u/14760...,NaN,3.0,54951af4db8155e6700e15f6,NaN,NaN,/prodigeni,prodigeni,1.0,gi,558777eebb2c3e7c1586ccef,[],[],[],33,2015-06-22 02:50:22.006000+00:00,NaN,gi,False,[],1
149464,149464,NaT,NaN,https://avatars1.githubusercontent.com/u/14760...,https://avatars1.githubusercontent.com/u/14760...,NaN,3.0,54951af4db8155e6700e15f6,NaN,NaN,/prodigeni,prodigeni,1.0,*hi,558777f4bb2c3e7c1586ccf0,[],[],[],32,2015-06-22 02:50:28.810000+00:00,NaN,*hi,False,[],1
150788,150788,NaT,NaN,https://avatars1.githubusercontent.com/u/14760...,https://avatars1.githubusercontent.com/u/14760...,NaN,3.0,54951af4db8155e6700e15f6,NaN,NaN,/prodigeni,prodigeni,1.0,hi,55884eb03039387b15782e95,[],[],[],48,2015-06-22 18:06:40.387000+00:00,NaN,hi,False,[],1
152791,152791,NaT,NaN,https://avatars1.githubusercontent.com/u/14760...,https://avatars1.githubusercontent.com/u/14760...,NaN,3.0,54951af4db8155e6700e15f6,NaN,NaN,/prodigeni,prodigeni,1.0,could you please use a pastebin?,5588e9f7f207aa853a8c5fd4,[],[],[],36,2015-06-23 05:09:11.417000+00:00,NaN,could you please use a pastebin?,False,[],1


This null value on the `displayname` are relate to the same userid `54951af4db8155e6700e15f6.` 

We will check if we have another occurence of this id and if we can get the displayname from them.

In [30]:
#Get all rows with userid = 54951af4db8155e6700e15f6
clean_freecodecamp_casual.loc[clean_freecodecamp_casual['fromUser.id'] == '54951af4db8155e6700e15f6']

,Unnamed: 0,editedAt,fromUser.avatarUrl,fromUser.avatarUrlMedium,fromUser.avatarUrlSmall,fromUser.displayName,fromUser.gv,fromUser.id,fromUser.removed,fromUser.staff,fromUser.url,fromUser.username,fromUser.v,html,id,issues,mentions,meta,readBy,sent,status,text,unread,urls,v
149462,149462,NaT,NaN,https://avatars1.githubusercontent.com/u/14760...,https://avatars1.githubusercontent.com/u/14760...,NaN,3.0,54951af4db8155e6700e15f6,NaN,NaN,/prodigeni,prodigeni,1.0,gi,558777eebb2c3e7c1586ccef,[],[],[],33,2015-06-22 02:50:22.006000+00:00,NaN,gi,False,[],1
149464,149464,NaT,NaN,https://avatars1.githubusercontent.com/u/14760...,https://avatars1.githubusercontent.com/u/14760...,NaN,3.0,54951af4db8155e6700e15f6,NaN,NaN,/prodigeni,prodigeni,1.0,*hi,558777f4bb2c3e7c1586ccf0,[],[],[],32,2015-06-22 02:50:28.810000+00:00,NaN,*hi,False,[],1
150788,150788,NaT,NaN,https://avatars1.githubusercontent.com/u/14760...,https://avatars1.githubusercontent.com/u/14760...,NaN,3.0,54951af4db8155e6700e15f6,NaN,NaN,/prodigeni,prodigeni,1.0,hi,55884eb03039387b15782e95,[],[],[],48,2015-06-22 18:06:40.387000+00:00,NaN,hi,False,[],1
152791,152791,NaT,NaN,https://avatars1.githubusercontent.com/u/14760...,https://avatars1.githubusercontent.com/u/14760...,NaN,3.0,54951af4db8155e6700e15f6,NaN,NaN,/prodigeni,prodigeni,1.0,could you please use a pastebin?,5588e9f7f207aa853a8c5fd4,[],[],[],36,2015-06-23 05:09:11.417000+00:00,NaN,could you please use a pastebin?,False,[],1


This user appear just one day and maybe he didn't fill the displayname field. 

Since the username is very often used in our dataset as a concatenation of the display name, we can use the username of this user and fill in his displayname.

In [31]:
#Fill the displayname for this user
clean_freecodecamp_casual['fromUser.displayName'] = clean_freecodecamp_casual['fromUser.displayName'].fillna('Prodigeni')

In [32]:
#Sanity check
clean_freecodecamp_casual['fromUser.displayName'].isna().sum()

0

We confirm that there no more `null value` inside the `fromUser.displayName` column.

In [33]:
#An overview on column, which column contain null value?
i = 0
for col in clean_freecodecamp_casual.columns:
    if clean_freecodecamp_casual[col].isna().sum() > 0:
        i += 1
        print(f'{col} have {clean_freecodecamp_casual[col].isna().sum()} null value')
print(f'\n We have {i} columns containing null values.')

editedAt have 4931000 null value
fromUser.avatarUrl have 1827993 null value
fromUser.gv have 31830 null value
fromUser.removed have 5053983 null value
fromUser.staff have 5057366 null value
fromUser.v have 796548 null value
html have 19844 null value
status have 5041085 null value
text have 19558 null value

 We have 9 columns containing null values.


#### Column text - 19558 null value

This column contain text message that we will work on it. The null value correspond to about `0.4% of the rows.`

In [34]:
#Percentage               #Total notnull
(19558 / 5057400 ) * 100, 5057400 - 19558

(0.3867204492426939, 5037842)

The best way to deal with that is just to `Delete them.` Because deleting will not have any impact and we cannot fill them with any value to avoid risk of corrupt dataset.

So we delete any rows containing null value in columns `text.`

In [35]:
#Overview about these columns
clean_freecodecamp_casual[clean_freecodecamp_casual['text'].isna()]

,Unnamed: 0,editedAt,fromUser.avatarUrl,fromUser.avatarUrlMedium,fromUser.avatarUrlSmall,fromUser.displayName,fromUser.gv,fromUser.id,fromUser.removed,fromUser.staff,fromUser.url,fromUser.username,fromUser.v,html,id,issues,mentions,meta,readBy,sent,status,text,unread,urls,v
1104,1104,2015-01-02 02:50:01.283000+00:00,NaN,https://avatars0.githubusercontent.com/u/81359...,https://avatars0.githubusercontent.com/u/81359...,Camden,NaN,54a60717db8155e6700e4d39,NaN,NaN,/theCamDen,theCamDen,NaN,NaN,54a6073429aec36b6af1cc88,[],[],[],21,2015-01-02 02:49:24.680000+00:00,NaN,NaN,False,[],2
1954,1954,2015-01-03 00:58:21.509000+00:00,NaN,https://avatars1.githubusercontent.com/u/11469...,https://avatars1.githubusercontent.com/u/11469...,Dustin Ury,3.0,54a60826db8155e6700e4d3e,NaN,NaN,/teddy-error,teddy-error,4.0,NaN,54a73e97a8dd09551d4235f8,[],[],[],31,2015-01-03 00:57:59.357000+00:00,NaN,NaN,False,[],3
3701,3701,2015-01-04 05:16:57.972000+00:00,NaN,https://avatars0.githubusercontent.com/u/85534...,https://avatars0.githubusercontent.com/u/85534...,seahik,3.0,54a1cb32db8155e6700e3c72,NaN,NaN,/seahik,seahik,1.0,NaN,54a8cc6d91bb4a2a7b0bba86,[],[],[],15,2015-01-04 05:15:25.732000+00:00,NaN,NaN,False,[],2
3881,3881,2015-01-04 07:56:47.950000+00:00,NaN,https://avatars1.githubusercontent.com/u/52907...,https://avatars1.githubusercontent.com/u/52907...,Vais Mahendran,NaN,54a8f225db8155e6700e5893,NaN,NaN,/M-Vais,M-Vais,NaN,NaN,54a8f2303569bbe471f22b0d,[],[],[],30,2015-01-04 07:56:32.534000+00:00,NaN,NaN,False,[],2
4038,4038,2015-01-04 16:39:54.607000+00:00,NaN,https://avatars1.githubusercontent.com/u/70185...,https://avatars1.githubusercontent.com/u/70185...,Mark Howard,3.0,546fc6a7db8155e6700d6e87,NaN,NaN,/iheartkode,iheartkode,6.0,NaN,54a96cd43569bbe471f22eb7,[],[],[],25,2015-01-04 16:39:48.614000+00:00,NaN,NaN,False,[],2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5019053,1462853,2017-11-26 03:44:02.312000+00:00,https://avatars-05.gitter.im/gh/uv/4/wordyallen,https://avatars1.githubusercontent.com/u/45918...,https://avatars1.githubusercontent.com/u/45918...,Joe Narvaez,4.0,54b09f16db8155e6700e83d8,NaN,NaN,/wordyallen,wordyallen,37.0,NaN,5a1a3868ffa3e37919bdfcba,[],[],[],10,2017-11-26 03:43:36.185000+00:00,NaN,NaN,False,[],3
5024486,1468286,2017-11-28 16:48:23.983000+00:00,https://avatars-01.gitter.im/gh/uv/4/CEREBR4L,https://avatars2.githubusercontent.com/u/71539...,https://avatars2.githubusercontent.com/u/71539...,Moon Man,4.0,5696919516b6c7089cc2652e,NaN,NaN,/CEREBR4L,CEREBR4L,24.0,NaN,5a1d935487680e6230a6e928,[],[],[],12,2017-11-28 16:48:20.832000+00:00,NaN,NaN,False,[],2
5025994,1469794,2017-11-29 06:55:19.178000+00:00,https://avatars-03.gitter.im/gh/uv/4/yasserhus...,https://avatars2.githubusercontent.com/u/69832...,https://avatars2.githubusercontent.com/u/69832...,Yasser Hussain,4.0,56d93ae2e610378809c4f368,NaN,NaN,/yasserhussain1110,yasserhussain1110,66.0,NaN,5a1e59ccffa3e37919d26d9f,[],[],[],11,2017-11-29 06:55:08.045000+00:00,NaN,NaN,False,[],2
5037511,1481311,2017-12-02 18:20:19.873000+00:00,https://avatars-03.gitter.im/gh/uv/4/Linuslb,https://avatars2.githubusercontent.com/u/13337...,https://avatars2.githubusercontent.com/u/13337...,Linus,4.0,55a55af15e0d51bd787b5fa7,NaN,NaN,/Linuslb,Linuslb,5.0,NaN,5a22eebf3a80a84b5bab9914,[],[],[],6,2017-12-02 18:19:43.073000+00:00,NaN,NaN,False,[],2


In [36]:
#Deleting them
#Get all rows index
ind_tex = clean_freecodecamp_casual[clean_freecodecamp_casual['text'].isna()].index.values.tolist()
print(len(ind_tex))

#deleting all these index in my datasaet
# for i in ind_tex:
clean_freecodecamp_casual = clean_freecodecamp_casual.drop(ind_tex)

19558


In [37]:
#sanity check
clean_freecodecamp_casual['text'].isna().sum()

0

We confirm that there no more `null value` inside the `text` column.

#### Column html - 19844 null value

This column contain html message that we will work on it. It's look like a duplicated of `text column` but we will deal with this aspect later. The null value correspond to about `0.4%` of the rows.

In [38]:
#Percentage               #Total notnull
(19844 / 5057400 ) * 100, 5057400 - 19844

(0.39237552892790767, 5037556)

In [39]:
#Overview
clean_freecodecamp_casual['html'].isna().sum()

286

Look like after removing the null from `text` we reduce them in `html` too. So we can apply the same principe, **Delete all the null values.**

In [40]:
#Deleting them
#Get all rows index
ind_tex = clean_freecodecamp_casual[clean_freecodecamp_casual['html'].isna()].index.values.tolist()
print(len(ind_tex))

#deleting all these index in my datasaet
# for i in ind_tex:
clean_freecodecamp_casual = clean_freecodecamp_casual.drop(ind_tex)

286


In [41]:
#sanity check
clean_freecodecamp_casual['html'].isna().sum()

0

We confirm that there no more `null value` inside the `html` column.

In [42]:
#An overview on column, which column contain null value?
i = 0
for col in clean_freecodecamp_casual.columns:
    if clean_freecodecamp_casual[col].isna().sum() > 0:
        i += 1
        print(f'{col} have {clean_freecodecamp_casual[col].isna().sum()} null value')
print(f'\n We have {i} columns containing null values.')

editedAt have 4924716 null value
fromUser.avatarUrl have 1818159 null value
fromUser.gv have 31811 null value
fromUser.removed have 5034141 null value
fromUser.staff have 5037522 null value
fromUser.v have 793591 null value
status have 5021346 null value

 We have 7 columns containing null values.


In [43]:
#Head
clean_freecodecamp_casual.sample(5)

,Unnamed: 0,editedAt,fromUser.avatarUrl,fromUser.avatarUrlMedium,fromUser.avatarUrlSmall,fromUser.displayName,fromUser.gv,fromUser.id,fromUser.removed,fromUser.staff,fromUser.url,fromUser.username,fromUser.v,html,id,issues,mentions,meta,readBy,sent,status,text,unread,urls,v
2912046,1084046,NaT,https://avatars-04.gitter.im/gh/uv/3/Kiwilicious,https://avatars1.githubusercontent.com/u/20009...,https://avatars1.githubusercontent.com/u/20009...,Kiwilicious,3.0,583a51afd73408ce4f38df3b,NaN,NaN,/Kiwilicious,Kiwilicious,1.0,"<span data-link-type=""mention"" data-screen-nam...",584030ca8d65e3830ec43179,[],"[{'userIds': [], 'userId': '55bd66500fc9f982be...",[],30,2016-12-01 14:16:42.825000+00:00,NaN,"@texas2010 Thanks, was wondering how to get it...",False,[],1
582414,582414,NaT,NaN,https://avatars1.githubusercontent.com/u/15617...,https://avatars1.githubusercontent.com/u/15617...,@nViser,3.0,5637c02116b6c7089cb9376f,NaN,NaN,/nviser,nviser,2.0,<p>var Car = function() {<br> //Change this c...,56424a5104a883da3838e250,[],[],[],23,2015-11-10 19:49:37.622000+00:00,NaN,var Car = function() {\n //Change this constr...,False,[],1
5000482,1444282,NaT,https://avatars-01.gitter.im/gh/uv/4/Pethaf,https://avatars2.githubusercontent.com/u/13156...,https://avatars2.githubusercontent.com/u/13156...,Pethaf,4.0,55959cd515522ed4b3e3438b,NaN,NaN,/Pethaf,Pethaf,28.0,Just coding up a pre-made design?,5a1137c2540c78242d200c45,[],[],[],12,2017-11-19 07:50:26.602000+00:00,NaN,Just coding up a pre-made design?,False,[],1
2038242,210242,NaT,https://avatars-04.gitter.im/gh/uv/3/combsco,https://avatars2.githubusercontent.com/u/15504...,https://avatars2.githubusercontent.com/u/15504...,Chris,3.0,55b6fc710fc9f982beab34c5,NaN,NaN,/combsco,combsco,165.0,No one cares about your unicode,577a847eef3990705ba573d2,[],[],[],35,2016-07-04 15:45:02.167000+00:00,NaN,No one cares about your unicode,False,[],1
3818487,262287,NaT,https://avatars-05.gitter.im/gh/uv/4/luishendr...,https://avatars1.githubusercontent.com/u/60394...,https://avatars1.githubusercontent.com/u/60394...,Luis Felipe López G.,4.0,55a098fd5e0d51bd787afe94,NaN,NaN,/luishendrix92,luishendrix92,99.0,ayyy,58e7b77468bee3091f1d6c92,[],[],[],13,2017-04-07 15:59:48.512000+00:00,NaN,ayyy,False,[],1


#### Column fromUser.gv - 31811 null value

We don't really at this moment understand what this column remain but it contains only `0.63%` null values.

In [44]:
#Percentage               #Total notnull
(31811 / 5057400 ) * 100, 5057400 - 31811

(0.628999090441729, 5025589)

In [45]:
#Overview on fromUser.gv column
clean_freecodecamp_casual['fromUser.gv'].value_counts()

3.0    3946072
4.0    1059666
Name: fromUser.gv, dtype: int64

We notice that it's only 2 values inside this column **[3 or 4].** So we can decide to `delete all the null values` because it will not impact the dataset.

In [46]:
#Deleting them
#Get all rows index
ind_tex = clean_freecodecamp_casual[clean_freecodecamp_casual['fromUser.gv'].isna()].index.values.tolist()
print(len(ind_tex))

#deleting all these index in my datasaet
# for i in ind_tex:
clean_freecodecamp_casual = clean_freecodecamp_casual.drop(ind_tex)

31811


In [47]:
#Sanity check
clean_freecodecamp_casual['fromUser.gv'].isna().sum()

0

We confirm that there no more `null value` inside the `fromUser.gv` column.

#### Column fromUser.avatarUrl  - 1818159  null value

This column contains urls redirecting to the image of each user. In this column we have `36%` of null values.


In [48]:
#Percentage               #Total notnull
(1818159 / 5057400 ) * 100, 5057400 - 1818159

(35.95046862023965, 3239241)

In [49]:
#Overview on fromUser.gv column
clean_freecodecamp_casual['fromUser.avatarUrl'].value_counts()

https://avatars-01.gitter.im/gh/uv/3/camperbot           251542
https://avatars-05.gitter.im/gh/uv/4/jodylecompte         83606
https://avatars-04.gitter.im/gh/uv/4/qmikew1              72182
https://avatars-05.gitter.im/gh/uv/3/apottr               65198
https://avatars-04.gitter.im/gh/uv/4/mstellaluna          63676
                                                          ...  
https://avatars-05.gitter.im/gh/uv/3/Nassaramuto              1
https://avatars-04.gitter.im/gh/uv/3/Jason-k83                1
https://avatars-03.gitter.im/gh/uv/3/nicuosan                 1
https://avatars-03.gitter.im/gh/uv/3/Dariuskr                 1
https://avatars-04.gitter.im/gh/uv/3/PatriciaMendozas         1
Name: fromUser.avatarUrl, Length: 137285, dtype: int64

In [50]:
#Overview on some sample row containing null value in fromUser.avatarUrl
clean_freecodecamp_casual[clean_freecodecamp_casual['fromUser.avatarUrl'].isna()].sample(5)

,Unnamed: 0,editedAt,fromUser.avatarUrl,fromUser.avatarUrlMedium,fromUser.avatarUrlSmall,fromUser.displayName,fromUser.gv,fromUser.id,fromUser.removed,fromUser.staff,fromUser.url,fromUser.username,fromUser.v,html,id,issues,mentions,meta,readBy,sent,status,text,unread,urls,v
243525,243525,NaT,NaN,https://avatars1.githubusercontent.com/u/98070...,https://avatars1.githubusercontent.com/u/98070...,Deiphage,3.0,55aebb240fc9f982beaa7c92,NaN,NaN,/deiphage,deiphage,NaN,Just read the code of conduct. Pretty awesome.,55aebf459402748911921f3b,[],[],[],41,2015-07-21 21:53:09.530000+00:00,NaN,Just read the code of conduct. Pretty awesome.,False,[],1
793501,793501,NaT,NaN,https://avatars1.githubusercontent.com/u/13561...,https://avatars1.githubusercontent.com/u/13561...,CamperBot,3.0,55b977f00fc9f982beab7883,NaN,NaN,/camperbot,camperbot,1.0,<blockquote>\nrphares sends brownie points to ...,5676b677091b6f9e0439e963,[],"[{'userIds': [], 'screenName': 'bvpenn'}]",[],50,2015-12-20 14:08:55.308000+00:00,NaN,> rphares sends brownie points to @bvpenn :spa...,False,[],1
301981,301981,NaT,NaN,https://avatars0.githubusercontent.com/u/11507...,https://avatars0.githubusercontent.com/u/11507...,genesisjoec,3.0,559ec5a715522ed4b3e3bf38,NaN,NaN,/genesisjoec,genesisjoec,NaN,(from texas),55c781857f578b465ff46ee4,[],[],[],28,2015-08-09 16:36:21.372000+00:00,NaN,(from texas),False,[],1
1244439,1244439,NaT,NaN,https://avatars1.githubusercontent.com/u/13352...,https://avatars1.githubusercontent.com/u/13352...,Serenity,3.0,55a7c9e08a7b72f55c3f991e,NaN,NaN,/qmikew1,qmikew1,69.0,and singing to them while doing the dishes,56d3c3e6649ae49a694b2d19,[],[],[],75,2016-02-29 04:07:02.709000+00:00,NaN,and singing to them while doing the dishes,False,[],1
1193095,1193095,NaT,NaN,https://avatars2.githubusercontent.com/u/11723...,https://avatars2.githubusercontent.com/u/11723...,Mark Canning,3.0,56c935e2e610378809c28ba9,NaN,NaN,/Maakux,Maakux,NaN,"They are the kind of people that say ‘well, it...",56c9df57833e58f50fb07728,[],[],[],74,2016-02-21 16:01:27.879000+00:00,NaN,"They are the kind of people that say ‘well, it...",False,[],1


The understanding of the dataset, allows us to assume that the columns `fromUser.avatarUrl, fromUser.avatarUrlMedium, fromUser.avatarUrlSmall` contain the urls to the user's images but at different sizes.

It's not really relevant for what we are doing to have all image size, so we can just deleting this column because it can be consider as a duplicate for `fromUser.avatarUrlMedium`

In [51]:
#Delete fromUser.avatarUrl column
clean_freecodecamp_casual = clean_freecodecamp_casual.drop(columns=['fromUser.avatarUrl'])

#sanity check
clean_freecodecamp_casual.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5005738 entries, 0 to 5057399
Data columns (total 24 columns):
 #   Column                    Dtype              
---  ------                    -----              
 0   Unnamed: 0                int64              
 1   editedAt                  datetime64[ns, UTC]
 2   fromUser.avatarUrlMedium  object             
 3   fromUser.avatarUrlSmall   object             
 4   fromUser.displayName      object             
 5   fromUser.gv               float64            
 6   fromUser.id               object             
 7   fromUser.removed          object             
 8   fromUser.staff            object             
 9   fromUser.url              object             
 10  fromUser.username         object             
 11  fromUser.v                float64            
 12  html                      object             
 13  id                        object             
 14  issues                    object             
 15  mentions       

In [52]:
#An overview on column, which column contain null value?
i = 0
for col in clean_freecodecamp_casual.columns:
    if clean_freecodecamp_casual[col].isna().sum() > 0:
        i += 1
        print(f'{col} have {clean_freecodecamp_casual[col].isna().sum()} null value')
print(f'\n We have {i} columns containing null values.')

editedAt have 4893102 null value
fromUser.removed have 5002337 null value
fromUser.staff have 5005711 null value
fromUser.v have 775307 null value
status have 4989563 null value

 We have 5 columns containing null values.


#### Column fromUser.removed  - 5002337  null value

This column contains the status of an user, if he was `removed` or not. In this column we have `99%` of null values.

In [53]:
#Percentage               #Total notnull
(5002337 / 5005738 ) * 100, 5005738 - 5002337

(99.93205797027332, 3401)

In [54]:
#Overview on the distribution on fromUser.removed
clean_freecodecamp_casual['fromUser.removed'].value_counts()

True    3401
Name: fromUser.removed, dtype: int64

We only have `True` value in our dataset concerning the `fromUser.removed` so we can assume that the null values correspond for us to `False` value.

We assume this null value is **False.**

For more usage, we will transform our `True/False` by `1/0` to have binary value to work with that.
- True => 1
- False/NaN => 0

In [55]:
#We transform our  value in this column
clean_freecodecamp_casual['fromUser.removed'] = np.where(clean_freecodecamp_casual['fromUser.removed'] == True, 1, 0)

In [56]:
#Sanity check
clean_freecodecamp_casual['fromUser.removed'].isna().sum()

0

In [69]:
#Sanity check
clean_freecodecamp_casual['fromUser.removed'].value_counts()

0    5002337
1       3401
Name: fromUser.removed, dtype: int64

We confirm that there no more `null value` inside the `fromUser.removed` column.

In [58]:
#An overview on column, which column contain null value?
i = 0
for col in clean_freecodecamp_casual.columns:
    if clean_freecodecamp_casual[col].isna().sum() > 0:
        i += 1
        print(f'{col} have {clean_freecodecamp_casual[col].isna().sum()} null value')
print(f'\n We have {i} columns containing null values.')

editedAt have 4893102 null value
fromUser.staff have 5005711 null value
fromUser.v have 775307 null value
status have 4989563 null value

 We have 4 columns containing null values.


In [60]:
# clean_freecodecamp_casual.info()

#### Column fromUser.staff  - 5005711  null value

This column contains the status of an user, if he was `staff member` or not. In this column we have `99%` of null values.

In [61]:
#Percentage               #Total notnull
(5005711 / 5005738 ) * 100, 5005738 - 5005711

(99.99946061899364, 27)

In [62]:
#Overview on the distribution on fromUser.removed
clean_freecodecamp_casual['fromUser.staff'].value_counts()

True    27
Name: fromUser.staff, dtype: int64

We only have `True` value in our dataset concerning the `fromUser.staff` so we can assume that the null values correspond for us to `False` value.

We assume this null value is **False.**

For more usage, we will transform our `True/False` by `1/0` to have binary value to work with that.
- True => 1
- False/NaN => 0

In [63]:
#We transform our  value in this column
clean_freecodecamp_casual['fromUser.staff'] = np.where(clean_freecodecamp_casual['fromUser.staff'] == True, 1, 0)

In [64]:
#Sanity check
clean_freecodecamp_casual['fromUser.staff'].isna().sum()

0

In [65]:
#Sanity check
clean_freecodecamp_casual['fromUser.staff'].value_counts()

0    5005711
1         27
Name: fromUser.staff, dtype: int64

We confirm that there no more `null value` inside the `fromUser.staff` column.

In [71]:
#An overview on column, which column contain null value?
i = 0
for col in clean_freecodecamp_casual.columns:
    if clean_freecodecamp_casual[col].isna().sum() > 0:
        i += 1
        print(f'{col} have {clean_freecodecamp_casual[col].isna().sum()} null value')
print(f'\n We have {i} columns containing null values.')

editedAt have 4893102 null value
fromUser.v have 775307 null value
status have 4989563 null value

 We have 3 columns containing null values.


#### Column status  - 4989563  null value

This column contains the status of an user, but it's not clairly mentionned which kind of status it is. In this column we have `99%` of null values.

In [73]:
#Percentage               #Total notnull
(4989563 / 5005738 ) * 100, 5005738 - 4989563

(99.67687082304347, 16175)

In [75]:
#Looking into the distribution
clean_freecodecamp_casual['status'].describe()

count     16175
unique        2
top        True
freq      12938
Name: status, dtype: object

This column doesn't contain enough data to work on it, because:
- We need a dataset with more than >50000 rows, but this column only have 16175 valide rows
- We cannot replace the null value by one of the possible value True/False, because it will change the distribution of this column in the dataset

So in conclusion we choose to remove the column from out dataset.

In [76]:
#Delete status column
clean_freecodecamp_casual = clean_freecodecamp_casual.drop(columns=['status'])

#sanity check
clean_freecodecamp_casual.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5005738 entries, 0 to 5057399
Data columns (total 23 columns):
 #   Column                    Dtype              
---  ------                    -----              
 0   Unnamed: 0                int64              
 1   editedAt                  datetime64[ns, UTC]
 2   fromUser.avatarUrlMedium  object             
 3   fromUser.avatarUrlSmall   object             
 4   fromUser.displayName      object             
 5   fromUser.gv               float64            
 6   fromUser.id               object             
 7   fromUser.removed          int64              
 8   fromUser.staff            int64              
 9   fromUser.url              object             
 10  fromUser.username         object             
 11  fromUser.v                float64            
 12  html                      object             
 13  id                        object             
 14  issues                    object             
 15  mentions       

We confirm that the `status` column no more exists in our dataset.

#### Column fromUser.v  - 775307  null value

This column contains numeric value but it's not clairly specify on what it is refer to. In this column we have `16%` of null values.

In [77]:
#Percentage               #Total notnull
(775307 / 5005738 ) * 100, 5005738 - 775307

(15.488365551692876, 4230431)

In [89]:
#What is the distribution
clean_freecodecamp_casual['fromUser.v'].value_counts()

1.0      728254
2.0      612751
3.0      249094
6.0      216635
4.0      167432
          ...  
229.0         1
393.0         1
140.0         1
244.0         1
131.0         1
Name: fromUser.v, Length: 219, dtype: int64

In [90]:
#What is the min and max
clean_freecodecamp_casual['fromUser.v'].describe().loc[['min', 'max']]

min       1.0
max    1410.0
Name: fromUser.v, dtype: float64

Because it's not clairly specify what this column represent, we can deal with the null value either by deleting them or by put a default value for them.

In this case, we are working with numeric value from `1 to 1410`, our default value could be **0**

In [91]:
#Replacing the null value by 0
clean_freecodecamp_casual['fromUser.v'] = clean_freecodecamp_casual['fromUser.v'].fillna(0)

In [95]:
#Sanity check
clean_freecodecamp_casual['fromUser.v'].isna().sum()

0

We confirm that there no more `null value` inside the `fromUser.v` column.

#### Column editedAt - 4931000 null value

It represent about **97%** of the rows in this column which contain `null value`.

How to deal with that?

In [96]:
#Percentage
(4931000 / 5005738 ) * 100, 5005738 - 4931000

(98.50695342025492, 74738)

In [ ]:
clean_freecodecamp_casual[clean_freecodecamp_casual['editedAt'].notnull()]

In [104]:
clean_freecodecamp_casual.head()

,Unnamed: 0,editedAt,fromUser.avatarUrlMedium,fromUser.avatarUrlSmall,fromUser.displayName,fromUser.gv,fromUser.id,fromUser.removed,fromUser.staff,fromUser.url,fromUser.username,fromUser.v,html,id,issues,mentions,meta,readBy,sent,text,unread,urls,v
0,0,NaT,https://avatars2.githubusercontent.com/u/21996...,https://avatars2.githubusercontent.com/u/21996...,Charles Watson,3.0,546fd99cdb8155e6700d6ec8,0,0,/sircharleswatson,sircharleswatson,1.0,no legumes either,54a4804fb2b2de4b786502bf,[],[],[],22,2014-12-31 23:01:35.647000+00:00,no legumes either,False,[],1
3,3,NaT,https://avatars2.githubusercontent.com/u/21996...,https://avatars2.githubusercontent.com/u/21996...,Charles Watson,3.0,546fd99cdb8155e6700d6ec8,0,0,/sircharleswatson,sircharleswatson,1.0,"<span data-link-type=""mention"" data-screen-nam...",54a480b89597baaa7dceceb2,[],"[{'userIds': [], 'userId': '54a47e0cdb8155e670...",[],22,2014-12-31 23:03:20.182000+00:00,@janetwalters008 It is. but it works. some peo...,False,[],1
4,4,NaT,https://avatars0.githubusercontent.com/u/42194...,https://avatars0.githubusercontent.com/u/42194...,Paul Gilliam,3.0,54a44bbbdb8155e6700e47de,0,0,/phgilliam,phgilliam,1.0,They guy that came up with the idea is kind of...,54a480ca9597baaa7dceceb5,[],[],[],22,2014-12-31 23:03:38.388000+00:00,They guy that came up with the idea is kind of...,False,[],1
6,6,NaT,https://avatars0.githubusercontent.com/u/42194...,https://avatars0.githubusercontent.com/u/42194...,Paul Gilliam,3.0,54a44bbbdb8155e6700e47de,0,0,/phgilliam,phgilliam,1.0,the*,54a480e229aec36b6af1bf45,[],[],[],23,2014-12-31 23:04:02.733000+00:00,the*,False,[],1
8,8,NaT,https://avatars2.githubusercontent.com/u/21996...,https://avatars2.githubusercontent.com/u/21996...,Charles Watson,3.0,546fd99cdb8155e6700d6ec8,0,0,/sircharleswatson,sircharleswatson,1.0,"<span data-link-type=""mention"" data-screen-nam...",54a480f99597baaa7dceceb9,[],"[{'userIds': [], 'userId': '54a44bbbdb8155e670...",[],23,2014-12-31 23:04:25.310000+00:00,@phgilliam I agree. he's pretty extreme lol,False,[],1


#### Empty coumn


In [110]:
clean_freecodecamp_casual['meta'].value_counts()

[]    5005738
Name: meta, dtype: int64

In [ ]:
clean_freecodecamp_casual['editedAt'].plot(grid=True)
plt.xlabel('Magnitude')
plt.ylabel('Date')
plt.title('Daily Trend for editedAt column')
plt.show()

In [ ]:
####count word
from sklearn.feature_extraction.text import CountVectorizer

# cd['html'] = clean_freecodecamp_casual['html'].copy()

cdi = pd.DataFrame({"text": clean_freecodecamp_casual['text'].copy(), "unread": clean_freecodecamp_casual['unread'].copy(),})


cdi = cdi.dropna()

cdi.isna().sum()

In [ ]:
cdi['unread'] = np.where(cdi['unread'] == False, 0, 1)

In [ ]:
# cdi['tet'] = cdi['text'].str.split(' ')
cdi.head()

In [ ]:
X = cdi['text']
y = cdi['unread']

In [ ]:
# splitting our data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.99, random_state=42)

In [ ]:
X_train.shape

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# 1. Instantiate 
bagofwords = CountVectorizer()

# 2. Fit 
bagofwords.fit(X_train)

# 3. Transform
small_transformed = bagofwords.transform(X_train)
small_transformed

In [ ]:
# how much memory does our sparse matrix take up?
(small_transformed.data.nbytes + small_transformed.indptr.nbytes + small_transformed.indices.nbytes)*8/1024/1024

In [ ]:
bagofwords.get_feature_names()

In [ ]:
small_transformed.toarray().sum(axis=0)

In [ ]:
word_counts = pd.DataFrame(
    {"counts": small_transformed.toarray().sum(axis=0)},
    index=bagofwords.get_feature_names()
).sort_values("counts", ascending=False)

word_counts.head(20).plot(kind="bar", figsize=(15, 5), legend=False)
plt.title("Top 20 most frequently occurring words")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.show()

<div id="container" style="position:relative;">
<div style="position:relative; float:right"><img style="height:25px""width: 50px" src ="https://drive.google.com/uc?export=view&id=14VoXUJftgptWtdNhtNYVm6cjVmEWpki1" />
</div>
</div>